<a href="https://colab.research.google.com/github/linkvarun/Jupyter_Notebook/blob/master/Optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import  Adadelta, Adam, RMSprop, Adagrad, Nadam, Adamax
from tensorflow.keras.optimizers.experimental import SGD

SEED = 7

In [ ]:
data = pd.read_csv('/content/winequality-red.csv', sep=",") # this could be used for classification or numerical regression
print(data.head())
print(len(data))
y = data['quality']
X = data.drop(['quality'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [ ]:
def create_model(opt):
    model = Sequential()
    model.add(Dense(100, input_dim=X_train.shape[1],
    activation='relu',kernel_initializer='glorot_normal'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation='tanh', activity_regularizer='L2'))
    model.add(Dense(25, activation='relu', activity_regularizer='L2'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='tanh', activity_regularizer='L2'))
    model.add(Dense(1, activation='linear'))
    return model

In [ ]:
def create_callbacks(opt):
    callbacks = [
    EarlyStopping(monitor='mean_squared_error', min_delta = 0.0001, patience=5, verbose=1),
    ModelCheckpoint('optimizers_best_' + opt + '.h5', monitor='mean_squared_error', save_best_only=True, verbose=1)
    ]
    return callbacks

In [ ]:
# add momentum, adagrad, nesterov
opts = dict({
     #'sgd': SGD(),
     'sgd-001': SGD(learning_rate=0.001, weight_decay=0.00001),
     'sgd-momentum': SGD(learning_rate=0.0001, momentum=0.3, nesterov= True),
     'adam': Adam(),
     'adadelta': Adadelta(),
     'rmsprop': RMSprop(),
     'rmsprop-0001': RMSprop(learning_rate=0.0001),
     'nadam': Nadam(),
     'adamax': Adamax()
    })

In [ ]:
batch_size = 64
n_epochs = 1000

results = []
# Loop through the optimizers

In [ ]:
for opt in opts:
    model = create_model(opt)
    callbacks = create_callbacks(opt)
    model.compile(loss='mse', optimizer=opts[opt], metrics=['mean_squared_error'])
    hist = model.fit(X_train.values, y_train, batch_size=batch_size, epochs=n_epochs, validation_data=(X_val.values, y_val), verbose=1,
    callbacks=callbacks)
    best_epoch = np.argmin(hist.history['mean_squared_error'])
    best_acc = hist.history['mean_squared_error'][best_epoch]
    best_model = create_model(opt)

    # Load the model weights with the highest validation accuracy
    best_model.load_weights('optimizers_best_' + opt + '.h5')
    best_model.compile(loss='mse', optimizer=opts[opt], metrics=['mean_squared_error'])
    score = best_model.evaluate(X_test.values, y_test, verbose=1)
    results.append([opt, best_epoch, best_acc, score[1]])

Epoch 1/1000
13/16 [=======================>......] - ETA: 0s - loss: 18.8769 - mean_squared_error: 18.1779  
Epoch 1: mean_squared_error improved from inf to 16.61304, saving model to optimizers_best_sgd-001.h5
16/16 [==============================] - 7s 22ms/step - loss: 17.3305 - mean_squared_error: 16.6130 - val_loss: 6.0778 - val_mean_squared_error: 5.1935
Epoch 2/1000
14/16 [=========================>....] - ETA: 0s - loss: 7.1438 - mean_squared_error: 6.2883
Epoch 2: mean_squared_error improved from 16.61304 to 6.06680, saving model to optimizers_best_sgd-001.h5
16/16 [==============================] - 0s 9ms/step - loss: 6.9258 - mean_squared_error: 6.0668 - val_loss: 3.4936 - val_mean_squared_error: 2.5343
Epoch 3/1000
13/16 [=======================>......] - ETA: 0s - loss: 4.3951 - mean_squared_error: 3.4757
Epoch 3: mean_squared_error improved from 6.06680 to 3.34892, saving model to optimizers_best_sgd-001.h5
16/16 [==============================] - 0s 9ms/step - loss: 4.2

In [ ]:
! ls -l

total 1064
-rw-r--r-- 1 root root 143136 Jul 17 03:48 optimizers_best_adadelta.h5
-rw-r--r-- 1 root root 143136 Jul 17 03:49 optimizers_best_adamax.h5
-rw-r--r-- 1 root root 143136 Jul 17 03:48 optimizers_best_adam.h5
-rw-r--r-- 1 root root 143136 Jul 17 03:49 optimizers_best_nadam.h5
-rw-r--r-- 1 root root 102224 Jul 17 03:49 optimizers_best_rmsprop-0001.h5
-rw-r--r-- 1 root root 102224 Jul 17 03:48 optimizers_best_rmsprop.h5
-rw-r--r-- 1 root root 102224 Jul 17 03:48 optimizers_best_sgd-001.h5
-rw-r--r-- 1 root root 102224 Jul 17 03:48 optimizers_best_sgd-momentum.h5
drwxr-xr-x 1 root root   4096 Jul 13 13:33 sample_data
-rw-r--r-- 1 root root 100951 Jul 17 03:13 winequality-red.csv


In [ ]:
res = pd.DataFrame(results)
res.columns = ['optimizer', 'epochs', 'val_mean_squared_error', 'mean_squared_error']
res

,optimizer,epochs,val_mean_squared_error,mean_squared_error
0,sgd-001,16,0.757509,0.602991
1,sgd-momentum,42,0.985794,0.569350
2,adam,33,0.675962,0.577201
3,adadelta,33,32.755257,32.006981
4,rmsprop,20,0.640166,0.540763
5,rmsprop-0001,144,0.630730,0.549706
6,nadam,35,0.668086,0.554688
7,adamax,28,0.724924,0.584273


In [ ]:
import tensorflow as tf
initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=1.)
values = initializer(shape=(7,7))
print(values)

tf.Tensor(
[[ 0.5033965   0.0339929  -0.11890381 -0.31825864 -0.70629233  1.0356468
  -0.1009609 ]
 [ 0.46455902  0.6481436  -0.7725586   0.4411914   0.40547988  0.9479252
   0.65615755]
 [ 0.6783782   0.13453487  0.10115083 -0.9447473   1.5691861  -0.3007577
   0.71164674]
 [ 0.8791011  -0.08741058 -1.2740767   0.16965912 -1.9728805  -0.55733126
  -1.0780123 ]
 [ 0.09958404  0.70558786  0.03193973 -0.01851057  0.49650174 -0.10525787
   0.25365728]
 [-1.0209355   0.38817742  1.0137835   1.3609387   0.09022329  0.3573908
   0.6081653 ]
 [-0.73727465  0.57941383  0.57129896  0.57133746 -1.0857868  -0.35548192
  -0.71652466]], shape=(7, 7), dtype=float32)


In [ ]:
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
values = initializer(shape=(7,7))
print(values)

tf.Tensor(
[[ 8.39114666e-01 -6.57521546e-01 -2.35330582e+00 -5.64857900e-01
   1.26034653e+00 -7.40655482e-01  1.95219862e+00]
 [ 4.05456161e+00  1.18379378e+00 -1.27772462e+00 -1.26333439e+00
   4.39128906e-01 -4.47810501e-01  6.98302865e-01]
 [ 1.10090666e-01 -6.07119203e-01  2.49132410e-01 -1.28387630e-01
   1.86336327e+00 -1.05968976e+00 -3.04458797e-01]
 [ 7.09035456e-01  6.75821066e-01  1.06326234e+00  6.74264610e-01
  -1.16655803e+00  7.91552246e-01 -1.39003372e+00]
 [ 1.77362633e+00  1.27789891e+00  1.39691517e-01 -6.53284013e-01
   1.17303634e+00  3.77456337e-01 -2.20998335e+00]
 [ 6.12794936e-01 -5.58811665e-01  1.78640798e-01 -1.37357712e+00
   1.32549763e+00  1.39280688e-03 -1.84837971e-02]
 [ 9.41408217e-01  6.56216681e-01 -3.07735831e-01  1.56174922e+00
   4.16374803e-01 -5.03594697e-01  5.81789970e-01]], shape=(7, 7), dtype=float32)
